# 第8章: ニューラルネット

第7章で取り組んだポジネガ分類を題材として、ニューラルネットワークで分類モデルを実装する。なお、この章ではPyTorchやTensorFlow、JAXなどの深層学習フレームワークを活用せよ。

In [1]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 19.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0

## 70. 単語埋め込みの読み込み

事前学習済み単語埋め込みを活用し、$|V| \times d_\rm{emb}$ の単語埋め込み行列$\pmb{E}$を作成せよ。ここで、$|V|$は単語埋め込みの語彙数、$d_\rm{emb}$は単語埋め込みの次元数である。ただし、単語埋め込み行列の先頭の行ベクトル$\pmb{E}_{0,:}$は、将来的にパディング（`<PAD>`）トークンの埋め込みベクトルとして用いたいので、ゼロベクトルとして予約せよ。ゆえに、$\pmb{E}$の2行目以降に事前学習済み単語埋め込みを読み込むことになる。

もし、Google Newsデータセットの[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)（300万単語・フレーズ、300次元）を全て読み込んだ場合、$|V|=3000001, d_\rm{emb}=300$になるはずである（ただ、300万単語の中には、殆ど用いられない稀な単語も含まれるので、語彙を削減した方がメモリの節約になる）。

また、単語埋め込み行列の構築と同時に、単語埋め込み行列の各行のインデックス番号（トークンID）と、単語（トークン）への双方向の対応付けを保持せよ。

In [1]:
import gensim.downloader as api

# Google Newsの学習済み単語ベクトルをダウンロード・ロード
w2v_model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [5]:
import numpy as np
import torch

# パラメータ
embedding_dim = 300  # demb
vocab_limit = 50000  # 使用する語彙数（+1して<PAD>の分も確保）

print('vocab size = ',len(w2v_model.index_to_key) + 1)

# 語彙を絞って使用（vocab_limit - 1）個取り出し（1個はPAD用）
vocab_words = w2v_model.index_to_key[:vocab_limit - 1]

# 単語からインデックスへの辞書（PAD: 0）
word_to_id = {"<PAD>": 0}
id_to_word = {0: "<PAD>"}

# 単語埋め込み行列の初期化（1行目は0ベクトル）
embedding_matrix = np.zeros((vocab_limit, embedding_dim), dtype=np.float32)

# 2行目以降を埋める
for i, word in enumerate(vocab_words, start=1):
    embedding_matrix[i] = w2v_model[word]
    word_to_id[word] = i
    id_to_word[i] = word

# PyTorchのテンソルに変換
E = torch.tensor(embedding_matrix)

print(f"Embedding matrix shape: {E.shape}")


vocab size =  3000001
Embedding matrix shape: torch.Size([50000, 300])


## 71. データセットの読み込み

[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されている[Stanford Sentiment Treebank (SST)](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip) をダウンロードし、訓練セット（train.tsv）と開発セット（dev.tsv）のテキストと極性ラベルと読み込み、全てのテキストをトークンID列に変換せよ。このとき、単語埋め込みの語彙でカバーされていない単語は無視し、トークン列に含めないことにせよ。また、テキストの全トークンが単語埋め込みの語彙に含まれておらず、空のトークン列となってしまう事例は、訓練セットおよび開発セットから削除せよ（このため、第7章の実験で得られた正解率と比較できなくなることに注意せよ）。

事例の表現方法は任意でよいが、例えば"contains no wit , only labored gags"がネガティブに分類される事例は、次のような辞書オブジェクトで表現すればよい。

```
{'text': 'contains no wit , only labored gags',
 'label': tensor([0.]),
 'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])}
```

この例では、`text`はテキスト、`label`は分類ラベル（ポジティブなら`tensor([1.])`、ネガティブなら`tensor([0.])`）、`input_ids`はテキストのトークン列をID列で表現している。

In [7]:
%%capture
!pip install scikit-learn
!pip install numpy
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
!unzip SST-2.zip

In [8]:
import pandas as pd
import torch

# SST-2の読み込み
train_df = pd.read_csv("SST-2/train.tsv", sep='\t')
dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t')

# 変換関数の定義
def convert_to_tensorized_data(df, word_to_id):
    processed = []
    for _, row in df.iterrows():
        text = row['sentence']
        label = torch.tensor([row["label"]], dtype=torch.float32)  # tensor([0.]) or tensor([1.])
        tokens = text.split()
        input_ids = [word_to_id[token] for token in tokens if token in word_to_id]
        if input_ids:  # 空リストでなければ処理
            input_tensor = torch.tensor(input_ids)
            processed.append({
                'text': text,
                'label': label,
                'input_ids': input_tensor
            })
    return processed

# 実行
train_tensor_data = convert_to_tensorized_data(train_df, word_to_id)
dev_tensor_data = convert_to_tensorized_data(dev_df, word_to_id)

# 確認（先頭の1件を表示）
print("Train size:", len(train_tensor_data))
print("Dev size:", len(dev_tensor_data))
print("Example (train):", train_tensor_data[0])


Train size: 65018
Dev size: 872
Example (train): {'text': 'hide new secretions from the parental units ', 'label': tensor([0.]), 'input_ids': tensor([ 5785,    66,    18,    12, 15095,  1594])}


## 72. Bag of wordsモデルの構築

単語埋め込みの平均ベクトルでテキストの特徴ベクトルを表現し、重みベクトルとの内積でポジティブ及びネガティブを分類するニューラルネットワーク（ロジスティック回帰モデル）を設計せよ。

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 平均埋め込みを使ったロジスティック回帰モデル
class MeanEmbeddingLogisticRegression(nn.Module):
    def __init__(self, embedding_matrix):
        super().__init__()
        # 事前学習済みの埋め込み行列を固定で読み込む
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True, padding_idx=0)
        self.linear = nn.Linear(embedding_matrix.size(1), 1)  # 入力次元 = 埋め込みの次元数

    def forward(self, input_ids):
        """
        input_ids: (batch_size, seq_len)
        """
        # 埋め込み取得: (batch_size, seq_len, emb_dim)
        embeddings = self.embedding(input_ids)

        # マスクして平均（PAD=0 を除く）
        mask = (input_ids != 0).unsqueeze(-1)  # (batch_size, seq_len, 1)
        masked_embeddings = embeddings * mask  # PAD部分を0にする
        sum_embeddings = masked_embeddings.sum(dim=1)
        lengths = mask.sum(dim=1).clamp(min=1)  # 0除算防止
        avg_embeddings = sum_embeddings / lengths  # (batch_size, emb_dim)

        # 線形層＋シグモイド
        logits = self.linear(avg_embeddings)  # (batch_size, 1)
        probs = torch.sigmoid(logits).squeeze(1)  # (batch_size)
        return probs


In [20]:
# 1. 事前学習済み埋め込みを NumPy で読み込んでいる場合
#    例: embedding_matrix.shape = (50000, 300)
# 2. PyTorch 用に変換
embedding_tensor = torch.tensor(embedding_matrix, dtype=torch.float32)

# 3. モデルに渡す
model = MeanEmbeddingLogisticRegression(embedding_tensor)

# 4. バッチをパディングしてモデルに渡す
sample_batch = torch.nn.utils.rnn.pad_sequence(
    [x['input_ids'] for x in train_tensor_data[:4]],
    batch_first=True,
    padding_value=0
)

# 5. 推論
pred_probs = model(sample_batch)
print(pred_probs)  # → shape: [4, 1]（バッチ内4事例のポジ確率）


tensor([0.5099, 0.5310, 0.4978, 0.4846], grad_fn=<SqueezeBackward1>)


## 73. モデルの学習

問題72で設計したモデルの重みベクトルを訓練セット上で学習せよ。ただし、学習中は単語埋め込み行列の値を固定せよ（単語埋め込み行列のファインチューニングは行わない）。また、学習時に損失値を表示するなど、学習の進捗状況をモニタリングできるようにせよ。

In [22]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

# 学習用データ（辞書のリスト）を PyTorch Dataset に変換
class SSTDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]['input_ids'], self.data[idx]['label']

    def __len__(self):
        return len(self.data)

# 修正された collate_fn（そのまま返すだけ）
def collate_no_padding(batch):
    input_ids_list, label_list = zip(*batch)
    return input_ids_list[0], label_list[0]  # バッチサイズ1想定

#PyTorch 用に変換
embedding_tensor = torch.tensor(embedding_matrix, dtype=torch.float32)
# モデル（embeddingはすでに提供されたもの）
model = MeanEmbeddingLogisticRegression(embedding_tensor)

# 埋め込みは凍結する
model.embedding.weight.requires_grad = False

# ハイパーパラメータ
epochs = 10
lr = 1e-3

# 損失関数と最適化手法
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# DataLoader 作成（バッチサイズ=1、パディングなし）
train_dataset = SSTDataset(train_tensor_data)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_no_padding)

# 学習ループ
for epoch in range(epochs):
    total_loss = 0.0
    model.train()
    for input_ids, label in train_loader:
        optimizer.zero_grad()
        input_ids = input_ids.unsqueeze(0)  # モデルはバッチ次元が必要
        output = model(input_ids).squeeze()
        loss = criterion(output, label.squeeze())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")


Epoch [1/10], Loss: 0.4374
Epoch [2/10], Loss: 0.4091
Epoch [3/10], Loss: 0.4066
Epoch [4/10], Loss: 0.4060
Epoch [5/10], Loss: 0.4055
Epoch [6/10], Loss: 0.4054
Epoch [7/10], Loss: 0.4052
Epoch [8/10], Loss: 0.4051
Epoch [9/10], Loss: 0.4051
Epoch [10/10], Loss: 0.4051


## 74. モデルの評価

問題73で学習したモデルの開発セットにおける正解率を求めよ。

In [28]:
from torch.utils.data import DataLoader

# 開発セット用 Dataset & DataLoader
dev_dataset = SSTDataset(dev_tensor_data)
dev_loader = DataLoader(dev_dataset, batch_size=1, shuffle=False, collate_fn=collate_no_padding)

# 評価モード
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for input_ids, labels in dev_loader:
        outputs = model(input_ids).squeeze()
        predicted = (outputs >= 0.5).float()  # 0.5 を閾値として 0 or 1 に変換
        labels = labels.squeeze(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Development Accuracy: {accuracy:.4f}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x5 and 300x1)

## 75. パディング

複数の事例が与えられたとき、これらをまとめて一つのテンソル・オブジェクトで表現する関数`collate`を実装せよ。与えられた複数の事例のトークン列の長さが異なるときは、トークン列の長さが最も長いものに揃え、0番のトークンIDでパディングをせよ。さらに、トークン列の長さが長いものから順に、事例を並び替えよ。

例えば、訓練データセットの冒頭の4事例が次のように表されているとき、

```
[{'text': 'hide new secretions from the parental units',
  'label': tensor([0.]),
  'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])},
 {'text': 'contains no wit , only labored gags',
  'label': tensor([0.]),
  'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])},
 {'text': 'that loves its characters and communicates something rather beautiful about human nature',
  'label': tensor([1.]),
  'input_ids': tensor([    4,  5053,    45,  3305, 31647,   348,   904,  2815,    47,  1276,  1964])},
 {'text': 'remains utterly satisfied to remain the same throughout',
  'label': tensor([0.]),
  'input_ids': tensor([  987, 14528,  4941,   873,    12,   208,   898])}]
```

`collate`関数を通した結果は以下のようになることが想定される。

```
{'input_ids': tensor([
    [     4,   5053,     45,   3305,  31647,    348,    904,   2815,     47,   1276,   1964],
    [  5785,     66, 113845,     18,     12,  15095,   1594,      0,      0,      0,      0],
    [   987,  14528,   4941,    873,     12,    208,    898,      0,      0,      0,      0],
    [  3475,     87,  15888,     90,  27695,  42637,      0,      0,      0,      0,      0]]),
 'label': tensor([
    [1.],
    [0.],
    [0.],
    [0.]])}
```


In [ ]:
# collate_fn: バッチ化＋パディング
def collate_batch(batch):
    input_ids_list, label_list = zip(*batch)
    padded_input_ids = nn.utils.rnn.pad_sequence(input_ids_list, batch_first=True, padding_value=0)
    labels = torch.stack(label_list)
    return padded_input_ids, labels


## 76. ミニバッチ学習

問題75のパディングの処理を活用して、ミニバッチでモデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

In [30]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

# 学習用データ（辞書のリスト）を PyTorch Dataset に変換
class SSTDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]['input_ids'], self.data[idx]['label']

    def __len__(self):
        return len(self.data)

#PyTorch 用に変換
embedding_tensor = torch.tensor(embedding_matrix, dtype=torch.float32)
# モデル（embeddingはすでに提供されたもの）
model2 = MeanEmbeddingLogisticRegression(embedding_tensor)

# 埋め込みは凍結する
model2.embedding.weight.requires_grad = False

# ハイパーパラメータ
batch_size = 64
epochs = 10
lr = 1e-3

# 損失関数と最適化手法
criterion = nn.BCELoss()
optimizer = optim.Adam(model2.parameters(), lr=lr)

# DataLoader の作成
train_dataset = SSTDataset(train_tensor_data)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)

# 学習ループ
for epoch in range(epochs):
    total_loss = 0.0
    model2.train()
    for input_ids, labels in train_loader:
        optimizer.zero_grad()
        outputs = model2(input_ids).squeeze()
        loss = criterion(outputs, labels.squeeze(1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * input_ids.size(0)

    avg_loss = total_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")


Epoch [1/10], Loss: 0.5503
Epoch [2/10], Loss: 0.4617
Epoch [3/10], Loss: 0.4373
Epoch [4/10], Loss: 0.4260
Epoch [5/10], Loss: 0.4196
Epoch [6/10], Loss: 0.4156
Epoch [7/10], Loss: 0.4127
Epoch [8/10], Loss: 0.4108
Epoch [9/10], Loss: 0.4094
Epoch [10/10], Loss: 0.4083


In [33]:
from torch.utils.data import DataLoader

# 開発セット用 Dataset & DataLoader
dev_dataset = SSTDataset(dev_tensor_data)
dev_loader = DataLoader(dev_dataset, batch_size=64, shuffle=False, collate_fn=collate_batch)

# 評価モード
model2.eval()

correct = 0
total = 0

with torch.no_grad():
    for input_ids, labels in dev_loader:
        outputs = model2(input_ids).squeeze()
        predicted = (outputs >= 0.5).float()  # 0.5 を閾値として 0 or 1 に変換
        labels = labels.squeeze(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Development Accuracy: {accuracy:.4f}")


Development Accuracy: 0.7706


## 77. GPU上での学習

問題76のモデル学習をGPU上で実行せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

## 78. 単語埋め込みのファインチューニング

問題77の学習において、単語埋め込みのパラメータも同時に更新するファインチューニングを導入せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

## 79. アーキテクチャの変更

ニューラルネットワークのアーキテクチャを自由に変更し、モデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。例えば、テキストの特徴ベクトル（単語埋め込みの平均ベクトル）に対して多層のニューラルネットワークを通したり、畳み込みニューラルネットワーク（CNN; Convolutional Neural Network）や再帰型ニューラルネットワーク（RNN; Recurrent Neural Network）などのモデルの学習に挑戦するとよい。